# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [81]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
driver = webdriver.Chrome()

In [82]:
#Accedemos a la web

URL = "https://ycharts.com/companies/TSLA/revenues"

driver.get(URL)

In [96]:
#Obtenemos el HTML
raw_data = driver.find_elements(By.TAG_NAME, "tr")

#Filtramos los datos que nos interesan

data = []
for i in raw_data:
    data.append(i.text)
    if ("General Motors Co" in str(i.text)):
        data.remove(i.text)
        break

#Cerramos conexion
driver.quit()

data

['',
 '',
 '',
 '',
 '',
 '',
 '',
 'Date Value',
 'June 30, 2024 25.50B',
 'March 31, 2024 21.30B',
 'December 31, 2023 25.17B',
 'September 30, 2023 23.35B',
 'June 30, 2023 24.93B',
 'March 31, 2023 23.33B',
 'December 31, 2022 24.32B',
 'September 30, 2022 21.45B',
 'June 30, 2022 16.93B',
 'March 31, 2022 18.76B',
 'December 31, 2021 17.72B',
 'September 30, 2021 13.76B',
 'June 30, 2021 11.96B',
 'March 31, 2021 10.39B',
 'December 31, 2020 10.74B',
 'September 30, 2020 8.771B',
 'June 30, 2020 6.036B',
 'March 31, 2020 5.985B',
 'December 31, 2019 7.384B',
 'September 30, 2019 6.303B',
 'June 30, 2019 6.35B',
 'March 31, 2019 4.541B',
 'December 31, 2018 7.226B',
 'September 30, 2018 6.824B',
 'June 30, 2018 4.002B',
 'Date Value',
 'March 31, 2018 3.409B',
 'December 31, 2017 3.288B',
 'September 30, 2017 2.985B',
 'June 30, 2017 2.790B',
 'March 31, 2017 2.696B',
 'December 31, 2016 2.285B',
 'September 30, 2016 2.298B',
 'June 30, 2016 1.270B',
 'March 31, 2016 1.147B',
 'Dec

In [100]:
#Limpiamos la data

for e in data:
    data.remove(e)
    if ("Date Value" in str(e)):
        data.remove(e)
        break
    
split_date = []
date = []
revenue = []
for i in range(len(data)):
    revenue.append (data[i].split(" ")[-1])
    split_date.append(data[i].split(" ")[:3])
    date.append(" ".join(split_date[i]))

data

['',
 'September 30, 2023 23.35B',
 'September 30, 2022 21.45B',
 'September 30, 2021 13.76B',
 'September 30, 2020 8.771B',
 'September 30, 2019 6.303B',
 'September 30, 2018 6.824B',
 'December 31, 2017 3.288B',
 'June 30, 2017 2.790B',
 'December 31, 2016 2.285B',
 'June 30, 2016 1.270B',
 'December 31, 2015 1.214B',
 'June 30, 2015 954.98M',
 'December 31, 2014 956.66M',
 'June 30, 2014 769.35M',
 'December 31, 2013 615.22M',
 'June 30, 2013 405.14M',
 'December 31, 2012 306.33M',
 'June 30, 2012 26.65M']

In [101]:
#Separamos los valores que queremos ingresar al DataFrame

split_date = []
date = []
revenue = []
for i in range(len(data)):
    revenue.append (data[i].split(" ")[-1])
    split_date.append(data[i].split(" ")[:3])
    date.append(" ".join(split_date[i]))

revenue

['',
 '23.35B',
 '21.45B',
 '13.76B',
 '8.771B',
 '6.303B',
 '6.824B',
 '3.288B',
 '2.790B',
 '2.285B',
 '1.270B',
 '1.214B',
 '954.98M',
 '956.66M',
 '769.35M',
 '615.22M',
 '405.14M',
 '306.33M',
 '26.65M']

In [102]:
#Creamos el Data Set
tesla = pd.DataFrame(zip(date, revenue), columns=["Date","Revenue"])

#Eliminamos la primera fila porque estaba en blanco.
tesla = tesla.drop(index=0)

tesla


,Date,Revenue
1,"September 30, 2023",23.35B
2,"September 30, 2022",21.45B
3,"September 30, 2021",13.76B
4,"September 30, 2020",8.771B
5,"September 30, 2019",6.303B
6,"September 30, 2018",6.824B
7,"December 31, 2017",3.288B
8,"June 30, 2017",2.790B
9,"December 31, 2016",2.285B
10,"June 30, 2016",1.270B
